## Ejercicios

Parte 1: Web Scraping con BeautifulSoup

Utilizando la biblioteca BeautifulSoup en Python, extrae información de la siguiente. Debes extraer la siguiente información:

- El nombre de las rutas que aparecen en la página web
- Dónde está ubicada la ruta
- El tipo de ruta. Si no está añadir "desconocido"
- Duración de la ruta
- Los kilómetros de la ruta
- La dificultad de la ruta
- El esfuerzo de la ruta
- La descripción de la ruta

In [1]:
# Librerías necesarias para resolver los ejercicios:

import os
from dotenv import load_dotenv


from datetime import datetime

from bs4 import BeautifulSoup
import requests

import pandas as pd

import re

pd.set_option('display.max_columns', None) # Visualizar todas las columnas de los DataFrames

In [64]:
# VERSIÓN MEJORADA SIGUIENDO INDICACIONES DADAS EN CLASE, DURANTE LA RESOLUCIÓN DE EJERCICIOS: 

def datos_rutas_montania(url):
    """
    Extrae información sobre rutas de montaña desde una URL dada.

    Args:
    url (str): La URL de la página web que contiene la información de las rutas de montaña.

    Returns:
    dict: Un diccionario que contiene la información de las rutas de montaña extraída de la página web.
    Cada clave del diccionario corresponde a un tipo de información, como 'nombre', 'donde', 'tipo', etc.
    """

    # Realizar una solicitud HTTP GET a la URL proporcionada
    respuesta = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if respuesta.status_code != 200:
        print(f"Parece que ha habido un problema con la extracción. El código de error es: {respuesta.status_code}")
    
    # Crear un objeto BeautifulSoup para analizar el contenido HTML de la respuesta
    sopa_rutas = BeautifulSoup(respuesta.content, 'html.parser')
    
    # Crear un diccionario para almacenar los resultados
    resultado = {
        'nombre': [],
        'donde': [],
        'tipo': [],
        'duracion': [],
        'km': [],
        'dificultad': [],
        'esfuerzo': [],
        'descripcion': []
    }
    
    # Extraer la información relevante de la página
    contenedores = sopa_rutas.find_all('div', {"class": "elementor-widget-container"})
    
    # Extraer y organizar la información
    lista_contenedores = [contenedor.text.strip() for contenedor in contenedores][15:54]
    resultado['nombre'] = [nombre.split(",")[0].replace('\n' , ', ') for nombre in lista_contenedores[::4]]
    resultado['descripcion'] = lista_contenedores[1::4]
    detalles_tecnicos = lista_contenedores[2::4]
    
    # Iterar sobre los detalles técnicos de las rutas
    for dato in detalles_tecnicos:
        
        # Dividir cada dato en líneas para su procesamiento
        dato_separado = dato.split('\n') 
        
        # Inicializar variables de estado para cada tipo de información
        donde_found = False
        tipo_found = False
        duracion_found = False
        km_found = False
        dificultad_found = False
        esfuerzo_found = False
        
        # Iterar por cada elemento de cada dato y buscar la información requerida
        for elemento in dato_separado:  
                       
            if re.search('^donde' , elemento.lower()):
                resultado['donde'].append(elemento.split(':')[1].replace("\xa0" , "").strip()) 
                donde_found = True
                
            if re.search('^tipo' , elemento.lower()):
                resultado['tipo'].append(elemento.split(':')[1].replace("\xa0" , "").strip().replace("." , "").capitalize())  
                tipo_found = True
                
            if re.search('^duración' , elemento.lower()):
                resultado['duracion'].append(elemento.split(':')[1].replace("\xa0" , "").strip().replace("." , "").capitalize())  
                duracion_found = True
                
            if re.search('^km' , elemento.lower()):
                resultado['km'].append(elemento.split(':')[1].replace("\xa0" , "").strip().replace("." , "").capitalize())  
                km_found = True
            
            if re.search('^dificultad' , elemento.lower()):
                resultado['dificultad'].append(elemento.split(':')[1].replace("\xa0" , "").strip().replace("." , "").capitalize())  
                dificultad_found = True
            
            if re.search('^esfuerzo' , elemento.lower()):
                resultado['esfuerzo'].append(elemento.split(':')[1].replace("\xa0" , "").strip().replace("." , "").capitalize())  
                esfuerzo_found = True
        
        # Si no se encontró información para un tipo específico después de iterar, se agrega 'desconocido' al resultado 
        if not donde_found:
            resultado['donde'].append('Desconocido')
        if not tipo_found:
            resultado['tipo'].append('Desconocido')
        if not duracion_found:
            resultado['duracion'].append('Desconocida')
        if not km_found:
            resultado['km'].append('Desconocidos')
        if not dificultad_found:
            resultado['dificultad'].append('Desconocida')
        if not esfuerzo_found:
            resultado['esfuerzo'].append('Desconocido')   
                
    return resultado

In [65]:
url_rutas = 'https://turismoactiva.com/rutas-de-montana-sierra-de-gredos/'

diccionario = datos_rutas_montania(url_rutas)

pd.DataFrame(diccionario)

,nombre,donde,tipo,duracion,km,dificultad,esfuerzo,descripcion
0,Pico Morenzon,"Sierra de Gredos, salida desde la Plataforma d...",Circular,4 horas,10,Baja (dependerá de la estación del año),Bajo – medio,"“Embárcate en la fascinante Ruta del Morezón, ..."
1,Pico de la Mira,"Sierra de Gredos, salida desde la Plataforma d...",Ida y vuelta,7 horas,16,Media (dependerá de la estación del año),Medio,Conquista el cielo en Gredos ascendiendo al Pi...
2,Pico Almanzor,"Sierra de Gredos, salida desde la Plataforma d...",Ida y vuelta,14 horas,22,Alta (dependerá de la estación del año),Alto,Desafía las alturas en el Circo de Gredos asce...
3,Laguna Grande de Gredos,"Sierra de Gredos, Plataforma de Gredos, sector...",Ida y vuelta,5 horas,12,Baja (dependerá de la estación del año),Bajo – medio,Una escapada hacia la Laguna Grande de Gredos....
4,Cinco Lagunas,"Sierra de Gredos, salida desde Navalperal del ...",Ida y vuelta,9 horas,25,Media (dependerá de la estación del año),Alto,Explora la mística Ruta de las Cinco Lagunas e...
5,Circo de Hoya Moros,"Sierra de Bejar, salida desde la Plataforma de...",Ida y vuelta (posibilidad de circular por la d...,7 horas,12,Media (dependerá de la estación del año),Medio,Adéntrate en la maravilla natural del Circo de...
6,Laguna de BarcoSierra de Gredos sector occidental,"Sierra de Gredos, aparcamiento carretera Nava ...",Ida y vuelta,7 horas,21,Baja (dependerá de la estación del año),Medio,Sumérgete en la serenidad de la naturaleza en ...
7,"Laguna de los Caballeros, Sierra de Gredos sec...","Sierra de Gredos, salida desde Navalguijo Sect...",Ida y vuelta,8 horas,25,Baja (dependerá de la estación del año),Alto,Adéntrate en la magia de la Sierra de Gredos e...
8,Laguna de la Nava,"Sierra de Gredos, salida desde la carretera de...",Ida y vuelta,8 horas,19,Medio (dependerá de la estación del año),Alto,La Laguna de la Nava es un fascinante paraje n...
9,"Tres Lagunas, La Nava","Sierra de Gredos, aparcamiento carretera Nava ...",Circular,"2 días, una noche",26,Alta (dependerá de la estación del año),Muy alto,La ruta más completa del sector occidental de ...


Parte 2: Obtención de Datos Climatológicos con la API de AEMET

Utiliza la API de AEMET para obtener información climatológica de la Sierra de Gredos. En concreto deberás usar el endpoint de 'predicciones-específicas' la prediccón de montaña para la sierra de gredos. Debe realizar las siguientes tareas:

- Incluir la temperatura máxima en la Sierra de Gredos
- Incluir la temperatura mínima en la Sierra de Gredos
- Incluir la fecha en la que se recopilaron los datos
- Incluir la sierra de donde vienen los datos

In [18]:
def extraccion_datos():
    """
    Esta función extrae la predicción de temperaturas mínimas y máximas para la Sierra de Gredos
    desde la API de AEMET y devuelve un diccionario con los datos extraídos.

    Returns:
        dict: Un diccionario con las predicciones de temperaturas mínimas y máximas,
              así como el lugar (Sierra de Gredos) y la fecha.

    """
    
    prediccion = {'temp_maxima' : [] , 'temp_minima' : [] , 'lugar' : [] , 'sierra' : [] , 'fecha' : []}
    
    load_dotenv()
    
    key = os.getenv("api_key")
    
    # URL de la API de AEMET para la predicción del tiempo actual
    url = f'https://opendata.aemet.es/opendata/api/prediccion/especifica/monta%C3%B1a/pasada/area/gre1/dia/0/?api_key={key}'
    
    # Realizar la solicitud GET a la URL de la API
    llamada = requests.get(url) 
    resultado = llamada.json()

    # Verificar si la solicitud fue exitosa
    if llamada.status_code != 200:
        print(f"Parece que ha habido un problema con la extracción. El código de error es: {llamada.status_code}")
        
    # Realizar una segunda solicitud GET para obtener los datos específicos
    llamada_datos = requests.get(resultado['datos'])
    
    # Verificar si la segunda solicitud fue exitosa
    if llamada_datos.status_code != 200:
        print(f"Parece que ha habido un problema con la extracción. El código de error es: {llamada_datos.status_code}")
        
    # Obtener la fecha y hora actual
    fecha_actual = datetime.now()

    # Formatear la fecha en el estilo AAAA-MM-DD
    fecha_formateada = fecha_actual.strftime("%Y-%m-%d")
        
    # Obtener los datos de temperatura mínima, máxima, nombre del lugar y fecha
    resultado_final = llamada_datos.json()
    datos_lugar = resultado_final[0]['seccion'][2]['lugar']
    
    # Iterar sobre los datos obtenidos y añadirlos al diccionario de predicción
    for dato in datos_lugar:
        prediccion['temp_maxima'].append(dato['maxima'])
        prediccion['temp_minima'].append(dato['minima'])
        prediccion['lugar'].append(dato['nombre'])
        prediccion['sierra'].append('Sierra de Gredos')
        prediccion['fecha'].append(fecha_formateada)  
    
    return prediccion

In [19]:
prediccion = extraccion_datos()

pd.DataFrame(prediccion)

,temp_maxima,temp_minima,lugar,sierra,fecha
0,-1,-6,Plataforma de Gredos,Sierra de Gredos,2024-02-27
1,6,-1,El Barco de Ávila,Sierra de Gredos,2024-02-27
2,5,-1,Piedrahíta,Sierra de Gredos,2024-02-27
3,-4,-9,La Serrota,Sierra de Gredos,2024-02-27
4,-2,-9,Pico Almanzor,Sierra de Gredos,2024-02-27
5,-3,-7,La Hoya (La Covatilla),Sierra de Gredos,2024-02-27


In [12]:
prediccion_media = {}

prediccion_media['temp_maxima'] = [round(sum(prediccion['temp_maxima'])/len(prediccion['temp_maxima']), 1)]

prediccion_media['temp_minima'] = [round(sum(prediccion['temp_minima'])/len(prediccion['temp_minima']), 1)]

prediccion_media['sierra'] = [prediccion['sierra'][0]]

prediccion_media['fecha'] = [prediccion['fecha'][0]]

pd.DataFrame(prediccion_media)

,temp_maxima,temp_minima,sierra,fecha
0,0.2,-5.5,Sierra de Gredos,2024-02-27
